In [12]:
import os
import csv
import json
import pandas as pd
from tqdm.notebook import tqdm

In [32]:
path_to_tweets = '../hierarchical_edge_bundling/tweets_marchandisation.csv'
path_to_users = 'following/'

df = pd.read_csv(path_to_tweets, sep=';')
sorted = df.sort_values(by="average_hash")

users_parsed = []
edges = "Source;Target;LinkType;Weight;AverageHash;Sha1;Category\n"
not_found = 0
hash_weight = 1
sha1_weight = 1
follow_weight = 1

current_ahash = sorted["average_hash"][0]
current_sha1 = sorted["sha1"][0]
current_users = []

In [33]:
# Créer des liens basés sur la copublication de average hashes et le follow

with tqdm(total=sorted.shape[0]) as pbar:
    # pour chaque ligne du csv
    for index, row in sorted.iterrows():
        user_name = row['from_user_name']
        ahash = row['average_hash']
        sha1 = row['sha1']
        category = row['category']

        # si le average hash est le même que celui d'avant
        if ahash == current_ahash:
            # on va checker tous les users d'avant qui avaient le même average hash
            for current_user, current_sha1 in current_users:
                # on vérifie que ça n'est pas le même user, ni le même sha1
                if current_user != user_name and current_sha1 != sha1:
                    edges += f"{user_name};{current_user};Same_hash;{hash_weight};{ahash};null;{category}\n"
        # sinon on remet à 0 la liste des utilisateurs du averagehash
        else:
            current_users = []
            current_ahash = ahash

#        if sha1 == current_sha1:
#            for current_user, current_sha1 in current_users:
#                if current_user != user_name:
#                    edges += f"{user_name};{current_user};Same_sha1;{sha1_weight};{ahash};{sha1};{category}\n"
#        else:
#            current_users = []
#            current_sha1 = sha1

        # dans tous les cas on ajoute le user qu'on est en train de traiter avec son sha1 à la liste des users à comparer
        current_users.append([user_name, sha1])

        try:
            # si on a pas déjà regardé les followers de l'utilisateur
            if user_name not in users_parsed:
                # on ouvre le fichier des followers
                with open(path_to_users + user_name + '.json', 'r') as f:
                    # on ne garde que les followers qui sont aussi dans le corpus
                    data = [user['username'] for user in json.load(
                        f) if user['username'] in list(df['from_user_name'])]
                    # on créé un lien entre chacun de ces followers et l'utilisateur courant
                    for user in data:
                        edges += f"{user_name};{user};Follow;{follow_weight};null;null;null\n"
                users_parsed.append(user_name)
        except FileNotFoundError:
            print(user_name + " not found!")
            not_found += 1
        except json.JSONDecodeError:
            print(user_name + " error!")
            not_found += 1
        pbar.update(1)


  0%|          | 0/1986 [00:00<?, ?it/s]

In [34]:
# Écrire le .csv

with open(f"fol_{follow_weight}_ahash_{hash_weight}_sha1_{sha1_weight}.csv", 'w', encoding='utf-8') as f:
    f.write(edges)